In [1]:
import pandas as pd
import numpy as np

In [2]:
#reading datasets
dataset = pd.read_csv('dataset_full.csv')
test = pd.read_csv('test.csv')

In [3]:
#Tratando missings
dataset['NSE_RIMAC'].fillna('Z',inplace=True)
dataset['SEGMENTO_INTERNO'].fillna('MADERA', inplace=True)
dataset['CANALDES'].fillna('CORREDOR',inplace=True)
dataset['CLASEVEHDES'].fillna('OTRO',inplace=True)
dataset['USOVEHDES'].fillna('OTRO',inplace=True)

marca_auto = ['TOYOTA', 'HYUNDAI','KIA', 'NISSAN', 'SUZUKI', 'VOLKSWAGEN', 'CHEVROLET','HONDA', 'MAZDA',
              'SUBARU', 'RENAULT', 'BMW', 'MITSUBISHI', 'JEEP', 'PEUGEOT', 'FORD', 'AUDI', 'MERCEDES BENZ', 
              'VOLVO', 'GREAT WALL', 'JAC', 'DAIHATSU', 'SSANGYONG', 'CHERY', 'CITROEN', 'SEAT', 'DODGE', 
              'SSANG YONG', 'FOTON', 'ISUZU', 'HINO', 'FIAT', 'BYD', 'MG', 'CHANGAN', 'INTERNATIONAL', 
              'BRILLIANCE', 'CHANGHE', 'MINI', 'LAND ROVER', 'PORSCHE', 'GEELY', 'DONG FENG', 'MAHINDRA',
              'LIFAN', 'JINBEI', 'DAEWOO', 'BRIDA', 'FREIGHTLINER', 'BAW', 'HAFEI', 'BAIC YINXIANG',
              'MITSUBISHI FUSO', 'FAW', 'DFSK', 'BAJAJ', 'KENWORTH', 'BAIC', 'LEXUS', 'ZOTYE', 'YUEJIN',
              'TVS', 'SCANIA', 'FORLAND', 'HAIMA', 'JMC', 'GONOW']

dataset['MARCAVEHDES'].fillna('OTRO', inplace=True)
dataset.loc[~dataset['MARCAVEHDES'].isin(marca_auto),'MARCAVEHDES'] = 'OTRO'

var_numeric = ['NUM_PRODUCTOS', 'OK', 'CPP', 'DEFICIENTE', 'DUDOSO', 'PERDIDA',
               'SALDO_SBS', 'SALDO_TC_SBS', 'SALDO_VEH_SBS', 'SALDO_HIP_SBS',
               'SALDO_PP_SBS', 'SALDO_MICRO_SBS', 'SALDO_PEQUENA_SBS','SALDO_OTROS', 'SUMASEG', 
               'NRO_SINIESTROS', 'MTO_SINIESTROS', 'DEUDA_HIINI']

for var in var_numeric:
    dataset[var].fillna(0, inplace=True)

In [4]:
#Filtering dataset Periodo > 201606
dataset = dataset.loc[dataset['PERIODO']>201606]

In [5]:
#feature engineering
dataset['anho'] = dataset['PERIODO']/100
dataset['antiguedad_auto'] = dataset['anho'] - dataset['ANOFABRIC']
dataset['antiguedad_auto'].fillna(0, inplace=True)

dataset['FEC_NACIMIENTO'] = pd.to_datetime(dataset['FEC_NACIMIENTO'], format="%d/%m/%Y")
dataset['FEC_ACTUAL'] = pd.to_datetime((dataset['PERIODO']*100)+28, format="%Y%m%d")
dataset['FECHA_FIN'] = pd.to_datetime(dataset['FECHA_FIN'])
dataset['FECHA_INICIO'] = pd.to_datetime(dataset['FECHA_INICIO'])

dataset['edad'] = dataset['anho'] - dataset['FEC_NACIMIENTO'].dt.year
dataset['duracion'] = (dataset['FECHA_FIN'] - dataset['FECHA_FIN']).astype('timedelta64[D]')
dataset['antiguedad'] = (dataset['FEC_ACTUAL'] - dataset['FECHA_INICIO']).astype('timedelta64[D]')

dataset["Suma_Deuda_3_meses"] = dataset.groupby(["CUC"])["Deuda"].shift(1).fillna(0) + \
dataset.groupby(["CUC"])["Deuda"].shift(2).fillna(0) + \
dataset.groupby(["CUC"])["Deuda"].shift(3).fillna(0)

dataset["Deuda_3_meses"] = 0
dataset.loc[dataset["Suma_Deuda_3_meses"] > 0,"Deuda_3_meses"] = 1

dataset["Suma_Deuda_6_meses"] = dataset.groupby(["CUC"])["Deuda"].shift(1).fillna(0) + \
dataset.groupby(["CUC"])["Deuda"].shift(2).fillna(0) + \
dataset.groupby(["CUC"])["Deuda"].shift(3).fillna(0)+ \
dataset.groupby(["CUC"])["Deuda"].shift(4).fillna(0) + \
dataset.groupby(["CUC"])["Deuda"].shift(5).fillna(0) + \
dataset.groupby(["CUC"])["Deuda"].shift(6).fillna(0)

dataset["Deuda_6_meses"] = 0
dataset.loc[dataset["Suma_Deuda_6_meses"] > 0,"Deuda_6_meses"] = 1

In [6]:
nse_dict = {'A':10,'A1': 9, 'A2':8, 'B1':7, 'B2':6, 'C1':5, 'C2':4, 'D1':3, 'D2':2, 'E':1}
dataset["new_nse"] = dataset["NSE_RIMAC"].map(nse_dict)
dataset.sort_values(["PERIODO"], inplace=True)
dataset["change_nse"] = dataset["new_nse"] - dataset.groupby(["CUC"])["new_nse"].shift(1)

In [7]:
dataset.fillna(0, inplace=True)

In [8]:
#discrete to numerical
var_discretas = ['NSE_RIMAC', 'CANALDES', 'CLASEVEHDES', 'MARCAVEHDES', 'USOVEHDES', 'SEGMENTO_INTERNO',
                'SEXO', 'UBICACION', 'DEPARTAMENTO']
for var in var_discretas:
    tmp = dataset.groupby([var], as_index=False)['Target'].mean()
    tmp = tmp.sort_values('Target', ascending=False).reset_index()
    dic_var = {}
    for i in tmp.index.values:
        dic_var[tmp.ix[i,var]] = i
    dataset[var] = dataset[var].map(lambda x:dic_var[x])
    

In [9]:
#more feature engineering
prod_vehi = dataset.groupby(["PERIODO","CUC"], as_index=False).agg({'P_AUTO':'nunique'})
dataset = dataset.merge(prod_vehi, how='left', on=['PERIODO', 'CUC'])

In [10]:
#test set
test_tmp = dataset.loc[(dataset['PERIODO'] == 201612) & (dataset['Deuda'] ==0), ]
test_fin = test_tmp.merge(test, how='inner', on='ID_UNICO')
print test_tmp.shape

(48915, 81)


In [11]:
#train set
train = dataset.loc[(dataset['Deuda']==0) & (dataset['PERIODO'] != 201612),]

In [12]:
#predictores
no_var = ['PERIODO', 'ANOFABRIC', 'FECHA_FIN', 'FECHA_INICIO','CUC', 'P_AUTO_x', 'ID_UNICO', 'ID_POLIZA',
          'FEC_NACIMIENTO', 'UBIGEO', 'NUM_FACTURAS', 'MONTO_DOLARES', 'Deuda', 
          'new_Deuda', 'Target', 'PROBA', 'new_nse', 'anho', 'FEC_ACTUAL']

predictors = [var for var in dataset if var not in no_var ]

In [14]:
#train new customers
train_new = train.loc[train['antiguedad']<90,]
test_fin_new = test_fin.loc[test_fin['antiguedad']<90,]
print test_fin_new.shape

(6514, 82)


In [15]:
#test old customers
train_old = train.loc[train['antiguedad']>=90,]
test_fin_old = test_fin.loc[test_fin['antiguedad']>=90,]

In [16]:
#MODELO
import xgboost as xgb

params = {'objective': 'binary:logistic', 'silent': 1,
'max_depth': 15, 'eta': 0.4,
'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 1,
'max_delta_step': 0, 'gamma': 0, 'eval_metric': 'auc'}

def modeler(train, test, predictors, params, output):
    dtrain = xgb.DMatrix(train[predictors], label=train['Target'])
    dtest = xgb.DMatrix(test[predictors])
    watchlist = [(dtrain, 'dtrain')]
    num_rounds = 100
    preds = np.zeros(test.shape[0])
    for s in np.random.randint(0, 1000000, size=10):
        print s
        params['seed'] = s
        clf_xgb_main = xgb.train(dtrain=dtrain, params=params, num_boost_round=num_rounds, evals=watchlist, 
                                 verbose_eval=False, )
        preds += clf_xgb_main.predict(dtest)
    preds = preds/10
    test['PROBA'] = preds
    test[['ID_UNICO', 'PROBA']].to_csv(output)


In [17]:
modeler(train_new, test_fin_new, predictors, params, 'zgz_new.csv')
modeler(train_old, test_fin_old, predictors, params, 'zgz_old.csv')

293365
35592
518478
646406
548246
467962
282561
845775
776948
614597


/Users/puxama/.virtualenvs/env_models/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


946064
974806
666656
13595
332444
64694
343937
364189
580473
753492
